<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [124]:
import re
import json

no_ascii_char = ['é', '‟', 'ø', 'š', '«', '¸', '…', '\xad', '¬', 'θ', ';', '\x9d', '∝', '´', '¾', '€', '÷', '′', '„', 'в', '£', 'â', 'ƞ', 'ã', '≠', 'œ', '¼', '∏', '¢', '🙂', '“', 'с', '’', '∠', 'т', 'π', '¤', '·', '™', '⁄', '�', 'ª', '—', '∪', '•', '₹', '³', '‘', '¥', '½', 'δ', '∩', '²', 'ō', 'ƫ', 'ў', 'º', 'ʼ', 'æ', '∆', '≥', 'е', '×', '¯', '∗', 'ç', '\x81', '√', 'ï', 'ˆ', '¹', '‚', 'ɵ', '−', 'í', '°', '≤', '–', '♠', '”']

def get_number_mapping(input_list):
    sym2num = {}
    num2sym = {}
    for num_i in input_list:
        if num_i not in sym2num:
            sym_i = 'n{}'.format(len(sym2num))
            sym2num[sym_i] = num_i
            num2sym[num_i] = sym_i
    return sym2num, num2sym

def is_ascii(s):
    res = []
    for c in s:
        if ord(c) >= 128:
            res.append(c)
    return res

In [144]:
def preprocess(data_version = 'train'):
    with open('./{}.json'.format(data_version), 'r') as f:
        data = json.load(f)
    print(len(data))
    with open('./{}.txt'.format(data_version), 'w') as g:
        to_write = []
        for item in data:
            problem = str(item['Problem']).lower()
            problem = problem.replace(',', ' , ').replace('(', ' ( ').replace(')', ' ) ').replace('|', ' | ').strip().strip('|').strip()
            problem = ' ' + " ".join(problem.split()) + ' '
            
            result = str(item['linear_formula']).lower()
            result = result.replace(',', ' , ').replace('(', ' ( ').replace(')', ' ) ').replace('|', ' | ').strip().strip('|').strip()
            result = ' ' + " ".join(result.split()) + ' '
            numbers = re.findall(r"\d+\.?\d*", problem)
            sym2num, num2sym = get_number_mapping(numbers)
            for num_i in num2sym:
                if ' '+num_i+' ' in problem:
                    problem = problem.replace(' '+num_i+' ', ' '+num2sym[num_i]+' ')
                else:
                    raise RuntimeError()
            problem = problem.strip()
            result = result.strip()
            for c in no_ascii_char:
                problem = problem.replace(c, 'no_ascii_{}'.format(no_ascii_char.index(c)))
                result = result.replace(c, 'no_ascii_{}'.format(no_ascii_char.index(c)))
            to_write.append((problem,result))
        to_write = sorted(to_write, key=lambda x:len(x[0].split()))
        print('writing...')
        for data_i in to_write:
            g.write(data_i[0]+'\t'+data_i[1]+'\n')

In [148]:
preprocess('train')

29837
writing...


In [149]:
preprocess('dev')
preprocess('test')

4475
writing...
2985
writing...


In [152]:
arr_len = []
with open('train.txt', 'r') as f:
    for l in f:
        arr_len.append(len(l.strip().split('\t')[1].split()))
import numpy as np
print(np.max(arr_len))

384
